<div align="center" style=" font-size: 80%; text-align: center; margin: 0 auto">
<img src="https://raw.githubusercontent.com/Explore-AI/Pictures/master/Python-Notebook-Banners/Examples.png"  style="display: block; margin-left: auto; margin-right: auto;";/>
</div>

# Examples: Interactive visualisation in Python 



In this train, we will focus on creating unique and interactive visualisations. We will learn how to make our code block outputs interactive for the end user.

> ⚠️ **Note:** An active internet connection is needed to complete the exercises within this notebook. 

> ⚠️ **Note:** This exercise works best when executed in a Jupyter notebook.

## Learning objectives
By the end of this train, you should be able to:
* Create advanced custom visualisations in Python.
* Create interactive visualisations in a Jupyter notebook.


## Introduction

Sometimes we require interaction in our Jupyter notebooks rather than a PowerBI dashboard. A potential reason would be to employ more sophisticated technology to meet typical business intelligence use cases. Traditional BI tools are ideal for creating dashboards created from SQL or CSV data. However, when we try to display information created by more sophisticated logic, we often fall short.

With interactive widgets in a notebook, we can leverage the full power of Python to express calculations and build visualisations while exposing "our logic" to the end user, allowing them to modify aspects of the visualisation. We may either add a few interactive controls and plots to notebooks or build completely functional applications and interactive dashboards. Both can be created with components from the built-in core widgets, such as `buttons`, `sliders`, and `dropdowns`, or with the rich ecosystem of custom widget libraries based on the Jupyter widgets framework, such as interactive maps with `ipyleaflet` or `2-D plots` with `bqplot`.

## Getting the notebook ready
Let's ensure we have the relevant packages installed.

### Step 1: Install the relevant packages

In [1]:
pip install scipy

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install ipywidgets


Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install chart-studio

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install pyarrow

Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install cufflinks

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for cufflinks: filename=cufflinks-0.17.3-py3-none-any.whl size=68721 sha256=2c60dba7800571972f7e39eed3d18271f12af7145ba40c0b7fe5661a90bf8330
  Stored in directory: c:\users\prisc\appdata\local\pip\cache\wheels\c0\d9\70\372130dacf508192607c1a7359c7bf0656b1a3b79f6cf66f7a
Successfully built cufflinks
Note: you may need to restart the kernel to use updated packages.


### Step 2: Enable interactive visualisations in Jupyter

In [6]:
!jupyter nbextension enable --py widgetsnbextension

usage: jupyter [-h] [--version] [--config-dir] [--data-dir] [--runtime-dir]
               [--paths] [--json] [--debug]
               [subcommand]

Jupyter: Interactive Computing

positional arguments:
  subcommand     the subcommand to launch

options:
  -h, --help     show this help message and exit
  --version      show the versions of core jupyter packages and exit
  --config-dir   show Jupyter config dir
  --data-dir     show Jupyter data dir
  --runtime-dir  show Jupyter runtime dir
  --paths        show all Jupyter paths. Add --json for machine-readable
                 format.
  --json         output paths as machine-readable json
  --debug        output debug information about paths

Available subcommands: console dejavu events execute kernel kernelspec lab
labextension labhub migrate nbconvert notebook qtconsole run script server
troubleshoot trust

Jupyter command `jupyter-nbextension` not found.


### Step 3: Import the required packages

In [7]:
# Standard data science helpers
import numpy as np
import pandas as pd
import scipy

# Instantiate the Plotly charting library.
import chart_studio.plotly as py
import plotly.graph_objs as go
import plotly.express as px
# We use plotly.offline as this allows us to create interactive
# visualisations without the use of an internet connection,
# making our notebook more distributable to others.
from plotly.offline import iplot, init_notebook_mode
init_notebook_mode(connected=True)

# The Cufflinks library allows us to directly bind
# Pandas DataFrames to Plotly charts.
import cufflinks as cf
# Once again, we use the Cufflinks library in offline mode.
cf.go_offline(connected=True)
cf.set_config_file(colorscale='plotly', world_readable=True)

# Extra options. We use these to make our interactive
# visualisations more aesthetically appealing.
from IPython.core.display import HTML
pd.options.display.max_rows = 30
pd.options.display.max_columns = 25

# Show all code cells outputs.
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

### Step 4: Read in the data

For this project, we'll work with some stats data from `medium.com`.

In [ ]:
df = pd.read_parquet('https://github.com/WillKoehrsen/Data-Analysis/blob/master/medium/data/medium_data_2019_01_26?raw=true')
df.head()

Let's have a quick descriptive look at the data we've just pulled: 

In [ ]:
df.describe()

## Interactive visualisations
We'll now work through a few examples of interactive visualisations. We'll start with a simple filter to see how the interface works.

### Example 1

In [ ]:
from ipywidgets import interact, interact_manual, widgets

To make a function interactive, we need to use the ```@interact``` decorator. 

But wait, what's a decorator? 

Simply put, a decorator is a Python function that wraps around another declared function to enhance or augment its behaviour, like putting a ribbon around a present to make it look prettier as a gift. We won't go into the semantics of how to _create_ a decorator, but you can easily spot them above a function with the form ``@decorator_name``. 
For more information, see: https://realpython.com/primer-on-python-decorators/

Consider the code below.

In [ ]:
@interact
def show_articles_more_than(column='claps', x=5000):
    display(HTML(f'<h4>Showing articles with more than {x} {column}<h4>'))
    display(df.loc[df[column] > x, ['title', 'claps', 'published_date', 'read_time', 'tags', 'views', 'reads']])

The ```@interact``` decorator automatically inferred that we wanted a text box for the column and an interactive slider. This decorator makes it incredibly simple to add interactivity. 

### Example 2

Now we will add a dropdown menu to select which column to filter on while simultaneously adding limits for x. 

A dropdown menu can be useful when we need to enforce constraints on the interaction.

In [ ]:
@interact
def show_titles_more_than(x=(10, 50000, 10),
                          column=['read_time', 'views', 'reads']):
    display(HTML(f'<h4>Showing articles with more than {x} {column}<h4>'))
    display(df.loc[df[column] > x, ['title', 'published_date', 'read_time', 'tags', 'views', 'reads']])

### Example 3

Moving on to more sophisticated data relationships that may be interactively represented, we construct a widget that quickly finds the correlation between two columns in the dataset.

In [ ]:
@interact
def correlations(column1=list(df.select_dtypes('number').columns),
                 column2=list(df.select_dtypes('number').columns)):
    print(f"Correlation: {df[column1].corr(df[column2])}")

Within the above code example, we need to specify Pandas columns that are of a numeric type using the code `df.select_dtypes('number')` because the correlation statistic is only defined between numeric values. 

### Example 4

We can use the same basic approach to create interactive widgets for plots. This approach allows us to expand the capabilities of the powerful `plotly` visualisation library.

In [ ]:
@interact
def scatter_plot(x=list(df.select_dtypes('number').columns),
                 y=list(df.select_dtypes('number').columns)[1:]):
    if x == y:
        print(f"Please select separate variables for X and Y")
    else:
        df.iplot(kind='scatter', x=x, y=y, mode='markers',
                 xTitle=x.title(), yTitle=y.title(), title=f'{y.title()} vs {x.title()}')
        ## If you are using Google Colab, comment out the above line of code and uncomment the lines below
        #fig = px.scatter(df, x=x, y=y, title=f'{y.title()} vs {x.title()}')
        #fig.show(renderer="colab")

### Example 5

In a similar fashion to the previous example, we can create a customisable heatmap of correlations:

In [ ]:
cscales = ['Greys', 'YlGnBu', 'Greens', 'YlOrRd', 'Bluered', 'RdBu',
            'Reds', 'Blues', 'Picnic', 'Rainbow', 'Portland', 'Jet',
            'Hot', 'Blackbody', 'Earth', 'Electric', 'Viridis', 'Cividis']

# We use the Figure Factory module of Plotly, which
# defines many unique and powerful plots to be used
# in Python.
# For more info, see: https://plot.ly/python/figure-factory-subplots/
import plotly.figure_factory as ff

corrs = df.corr(numeric_only=True)

@interact
def plot_corrs(colorscale=cscales):
    figure = ff.create_annotated_heatmap(z = corrs.round(2).values,
                                     x =list(corrs.columns),
                                     y=list(corrs.index),
                                     colorscale=colorscale,
                                     annotation_text=corrs.round(2).values)
    iplot(figure)
    ## If you are using Google Colab, comment out the above line of code and uncomment the line below
    #figure.show(renderer="colab")

### Example 6

Interaction can also be extremely useful when we want to try and view various configurations for a plot that we are trying to produce. 

The following code allows us to add some options to control the colour scheme of a given Plotly chart. Here we call the `@interact_manual` decorator, which allows us to select multiple variables before our chart is updated. 

In [ ]:
@interact_manual
def scatter_plot(x=list(df.select_dtypes('number').columns),
                 y=list(df.select_dtypes('number').columns)[1:],
                 theme=list(cf.themes.THEMES.keys()),
                 colorscale=list(cf.colors._scales_names.keys())):

    if x == y:
        print(f"Please select separate variables for X and Y")
    else:
        df.iplot(kind='scatter', x=x, y=y, mode='markers',
                 xTitle=x.title(), yTitle=y.title(),
                 text='title',
                 title=f'{y.title()} vs {x.title()}',
                theme=theme, colorscale=colorscale)
        ## If you are using Google Colab, comment out the above line of code and uncomment the line below
        #fig = px.scatter(df, x=x, y=y, title=f'{y.title()} vs {x.title()}')
        #fig.show(renderer="colab")

### Example 7

The decorator ```@interact``` is not the only way to use widgets. 

Read through the following example to figure out how else we can use widgets.

In [ ]:
df.set_index('published_date', inplace=True)

In [ ]:
def print_articles_published(start_date, end_date):
    start_date = pd.Timestamp(start_date)
    end_date = pd.Timestamp(end_date)
    stat_df = df.loc[(df.index >= start_date) & (df.index <= end_date)].copy()
    total_words = stat_df['word_count'].sum()
    total_read_time = stat_df['read_time'].sum()
    num_articles = len(stat_df)
    print(f'According to our dataset, published by Medium.com, there are {num_articles} articles between {start_date.date()} and {end_date.date()}.')
    print(f'These articles totalled {total_words:,} words and {total_read_time/60:.2f} hours to read.')

_ = interact(print_articles_published,
             start_date=widgets.DatePicker(value=pd.to_datetime('2018-01-01')),
             end_date=widgets.DatePicker(value=pd.to_datetime('2019-01-01')))

### Example 8

For our final example, we use the `Dropdown` and `DatePicker` widgets to plot one column cumulatively up to a certain time. 

In [ ]:
def plot_up_to(column, date):
    date = pd.Timestamp(date)
    plot_df = df.loc[df.index <= date].copy()
    plot_df[column].cumsum().iplot(mode='markers+lines',
                                   xTitle='published date',
                                   yTitle=column,
                                  title=f'Cumulative {column.title()} Until {date.date()}')

_ = interact(plot_up_to, column=widgets.Dropdown(options=list(df.select_dtypes('number').columns)),
             date = widgets.DatePicker(value=pd.to_datetime('2019-01-01')))

## Summary

Jupyter Notebook is a great data exploration and analysis environment. Using tools like notebook extensions and interactive widgets makes our job as data-driven storytellers even more efficient. Widgets are powerful tools that allow us to make our data more interactive and, therefore, more accessible to non-data scientists.

<div align="center" style=" font-size: 80%; text-align: center; margin: 0 auto">
<img src="https://raw.githubusercontent.com/Explore-AI/Pictures/refs/heads/master/ALX_banners/ALX_Navy.png"  style="width:140px";/>
</div>